# FINN - Functional Verification of End-to-End Flow
-----------------------------------------------------------------

**Important: This notebook depends on the [tfc_end2end_example](tfc_end2end_example.ipynb) notebook, because we are using models that are available at intermediate steps in the end-to-end flow. So please make sure the needed .onnx files are generated to run this notebook.**

In this notebook, we will show how to take the intermediate results of the end-to-end tfc example and verify their functionality with different methods. In the following picture you can see the section in the end-to-end flow about the *Simulation & Emulation Flows*. 

<img src="verification.svg" alt="Drawing" style="width: 500px;"/>

We will use the following helper functions, `showSrc` to show source code of FINN library calls and `showInNetron` to show the ONNX model at the current transformation step. The Netron displays are interactive, but they only work when running the notebook actively and not on GitHub (i.e. if you are viewing this on GitHub you'll only see blank squares).

In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showSrc, showInNetron
import os

build_dir = os.environ["FINN_BUILD_DIR"]

To verify the simulations, a "golden" output is calculated as a reference. This is calculated directly from the Brevitas model using PyTorch, by running some example data from the MNIST dataset through the trained model.

In [2]:
from pkgutil import get_data
import onnx
import onnx.numpy_helper as nph
import torch
from finn.util.test import get_test_model_trained

fc = get_test_model_trained("TFC", 1, 1)
raw_i = get_data("qonnx.data", "onnx/mnist-conv/test_data_set_0/input_0.pb")
input_tensor = onnx.load_tensor_from_string(raw_i)
input_brevitas = torch.from_numpy(nph.to_array(input_tensor).copy()).float()
output_golden = fc.forward(input_brevitas).detach().numpy()
output_golden

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1255: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1758.)
  return super(Tensor, self).rename(names)


array([[-1.4090618, -1.3267527,  0.9779036, -1.2444434, -1.4090618,
        -1.6559892, -1.3267527, -1.4913709, -1.3267527, -1.6559892]],
      dtype=float32)

## Simulation using Python <a id='simpy'></a>

If an ONNX model consists of [standard ONNX](https://github.com/onnx/onnx/blob/main/docs/Operators.md) nodes and/or FINN custom operations that do not belong to the fpgadataflow (`backend` $\neq$ `fpgadataflow.hls` or `backend` $\neq$ `fpgadataflow.rtl`) this model can be checked for functionality using Python.

To simulate a standard ONNX node [onnxruntime](https://github.com/microsoft/onnxruntime) is used. onnxruntime is an open source tool developed by Microsoft to run standard ONNX nodes. For the FINN custom op nodes execution, functions are defined. The following is an example of the execution function of an XNOR popcount node.


In [12]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau
%matplotlib inline

In [13]:
from qonnx.custom_op.general.xnorpopcount import xnorpopcountmatmul
showSrc(xnorpopcountmatmul)

def xnorpopcountmatmul(inp0, inp1):
    """Simulates XNOR-popcount matrix multiplication as a regular bipolar
    matrix multiplication followed by some post processing."""
    # extract the operand shapes
    # (M, K0) = inp0.shape
    # (K1, N) = inp1.shape
    K0 = inp0.shape[-1]
    K1 = inp1.shape[0]
    # make sure shapes are compatible with matmul
    assert K0 == K1, "Matrix shapes are not compatible with matmul."
    K = K0
    # convert binary inputs to bipolar
    inp0_bipolar = 2.0 * inp0 - 1.0
    inp1_bipolar = 2.0 * inp1 - 1.0
    # call regular numpy matrix multiplication
    out = np.matmul(inp0_bipolar, inp1_bipolar)
    # XNOR-popcount does not produce the regular dot product result --
    # it returns the number of +1s after XNOR. let P be the number of +1s
    # and N be the number of -1s. XNOR-popcount returns P, whereas the
    # regular dot product result from numpy is P-N, so we need to apply
    # some correction.
    # out = P-N
    # K = P+N
    # out + K = 

The function contains a description of the behaviour in Python and can thus calculate the result of the node.

This execution function and onnxruntime is used when `execute_onnx` from `onnx_exec` is applied to the model. The model is then simulated node by node and the result is stored in a context dictionary, which contains the values of each tensor at the end of the execution. To get the result, only the output tensor has to be extracted.

The procedure is shown below. We take the model right before the nodes should be converted into HW layers and generate an input tensor to pass to the execution function. The input tensor is generated from the Brevitas example inputs.

In [14]:
from torchvision import transforms
import numpy as np

class GetTransforms():
    '''Returns a list of transformations when type as requested amongst train/test
       Transforms('train') = list of transforms to apply on training data
       Transforms('test') = list of transforms to apply on testing data'''

    def __init__(self):
        pass

    def trainparams(self):
        train_transformations = [ #resises the image so it can be perfect for our model.
            transforms.RandomHorizontalFlip(), # FLips the image w.r.t horizontal axis
            transforms.RandomRotation((-7,7)),     #Rotates the image to a specified angel
            transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Set the color params
            transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
            transforms.Normalize((0.491, 0.482, 0.446), (0.247, 0.243, 0.261)) #Normalize all the images
            ]

        return train_transformations

    def testparams(self):
        test_transforms = [
            transforms.ToTensor(),
            transforms.Normalize((0.491, 0.482, 0.446), (0.247, 0.243, 0.261))
        ]
        return test_transforms

In [15]:
transformations = GetTransforms()
train_transforms = transforms.Compose(transformations.trainparams())
test_transforms = transforms.Compose(transformations.testparams())


class GetCIFAR10_TrainData():
    def __init__(self, dir_name:str):
        self.dirname = dir_name

    def download_train_data(self):
        return datasets.CIFAR10('resnet18/data', train=True, download=True, transform=train_transforms)

    def download_test_data(self):
        return datasets.CIFAR10('resnet18/data', train=False, download=True, transform=test_transforms)

In [16]:
import os
data = GetCIFAR10_TrainData(os.chdir(".."))
trainset = data.download_train_data()
testset = data.download_test_data()
trainloader = torch.utils.data.DataLoader(trainset, batch_size=592,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=592,
                                         shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

Target device: cpu


In [22]:
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper

model_for_sim = ModelWrapper("hardware/quantresnet18_weight2_files/quantresnet18_weight2_for_hw_conversion.onnx")

In [23]:
import torch
import numpy as np
import finn.core.onnx_exec as oxe

def inference_with_finn_onnx(current_inp: torch.Tensor, model_for_sim):
    """
    Run inference using a FINN-compatible ONNX model.

    Args:
        current_inp (torch.Tensor): Input tensor of shape (1, C, H, W).
        model_for_sim (ModelWrapper): Cleaned and transformed FINN ONNX model.

    Returns:
        logits (np.ndarray): Raw output from the model.
        predicted (np.ndarray): Predicted class index (argmax).
    """
    input_name = model_for_sim.graph.input[0].name
    output_name = model_for_sim.graph.output[0].name

    # Ensure correct input shape and type
    current_inp = current_inp.detach().cpu().numpy()
    if current_inp.ndim == 3:
        current_inp = np.expand_dims(current_inp, axis=0)  # Add batch dim
    current_inp = current_inp.astype(np.float32)

    # Build input dict and run inference
    input_dict = {input_name: current_inp}
    output_dict = oxe.execute_onnx(model_for_sim, input_dict)

    # Extract output and predicted class
    logits = output_dict[output_name]
    predicted = np.argmax(logits, axis=1)

    return logits, predicted

In [26]:
import torch

torch.manual_seed(42)

ok = 0
nok = 0
expected_batch_size = 592

for img, label_gt in testloader:
    img, label_gt = img.to(device), label_gt.to(device)
   
    if img.shape[0] == 0:
        continue  # skip empty batches

    original_batch_size = img.shape[0]

    # Pad or truncate to expected batch size
    if original_batch_size < expected_batch_size:
        padding_size = expected_batch_size - original_batch_size
        img = torch.cat([img, img[:padding_size]], dim=0)
        label_gt = torch.cat([label_gt, label_gt[:padding_size]], dim=0)
    elif original_batch_size > expected_batch_size:
        img = img[:expected_batch_size]
        label_gt = label_gt[:expected_batch_size]

  
    # FINN ONNX inference
    _, predicted_onnx = inference_with_finn_onnx(img, model_for_sim)
    predicted_onnx_tensor = torch.tensor(predicted_onnx, dtype=torch.long, device=device)


Exception: Shape mismatch for provided input global_in: found (1, 3, 32, 32) expected (592, 3, 32, 32) 

In [9]:
import finn.core.onnx_exec as oxe
input_dict = {"global_in": np.random.randint(0, 255, size=(1, 3, 32, 32), dtype=np.float64)}
output_dict = oxe.execute_onnx(model_for_sim, input_dict, return_full_exec_context=False)
output_pysim = output_dict[list(output_dict.keys())[0]]

#try:
#    assert np.isclose(output_pysim, np.where(output_golden[0]==np.amax(output_golden[0])), atol=1e-3).all()
#    print("Results are the same!")
#except AssertionError:
#    assert False, "The results are not the same!"

TypeError: Unsupported dtype dtype('float64') for randint

In [ ]:
output_dict

The result is compared with the theoretical "golden" value for verification.

## Simulation (cppsim) using C++

When dealing with HLS or RTL custom op nodes in FINN the simulation using Python is no longer sufficient. If the nodes are specialized to HLS layers, the simulation using C++ can be used. To do this, the input tensor is stored in a .npy file and C++ code is generated that reads the values from the .npy array, streams them to the corresponding `finn-hlslib` function and writes the result to a new .npy file. This in turn can be read in Python and processed in the FINN flow. For this example the model after setting the folding factors in the HLS variants of the layers, please be aware that this is not the full model, but the dataflow partition, so before executing at the end of this section we have to integrate the model back into the parent model.

<div class="alert alert-block alert-info">
<b>Note: HW layer can also be converted to RTL variants, in this case "cppsim" is not an option we can execute. If nevertheless "cppsim" is selected as execution mode for the layer, the execution defaults to the parent class. Like this, networks with a mix of HLS and RTL layers can be executed using "cppsim" for the HLS layers.</b> 
</div>

In [ ]:
model_for_cppsim = ModelWrapper(build_dir+"/tfc_w1_a1_set_folding_factors.onnx")

To generate the code for this simulation and to generate the executable two transformations are used:
* `PrepareCppSim` which generates the C++ code for the corresponding HLS layer
* `CompileCppSim` which compules the C++ code and stores the path to the executable

In [ ]:
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from qonnx.transformation.general import GiveUniqueNodeNames

model_for_cppsim = model_for_cppsim.transform(GiveUniqueNodeNames())
model_for_cppsim = model_for_cppsim.transform(PrepareCppSim())
model_for_cppsim = model_for_cppsim.transform(CompileCppSim())

When we take a look at the model using netron, we can see that the transformations introduced new attributes.

In [ ]:
model_for_cppsim.save(build_dir+"/tfc_w1_a1_for_cppsim.onnx")
#showInNetron(build_dir+"/tfc_w1_a1_for_cppsim.onnx")

The following node attributes have been added:
* `code_gen_dir_cppsim` indicates the directory where the files for the simulation using C++ are stored
* `executable_path` specifies the path to the executable

We take now a closer look into the files that were generated:

In [ ]:
from qonnx.custom_op.registry import getCustomOp

fc0 = model_for_cppsim.graph.node[1]
fc0w = getCustomOp(fc0)
code_gen_dir = fc0w.get_nodeattr("code_gen_dir_cppsim")
!ls {code_gen_dir}

Besides the .cpp file, the folder contains .h files with the weights and thresholds. The shell script contains the compile command and *node_model* is the executable generated by compilation. Comparing this with the `executable_path` node attribute, it can be seen that it specifies exactly the path to *node_model*.

To simulate the model the execution mode(exec_mode) must be set to "cppsim". This is done using the transformation SetExecMode.

In [ ]:
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode

model_for_cppsim = model_for_cppsim.transform(SetExecMode("cppsim"))
model_for_cppsim.save(build_dir+"/tfc_w1_a1_for_cppsim.onnx")

Before the model can be executed using `execute_onnx`, we integrate the child model in the parent model. The function reads then the `exec_mode` and writes the input into the correct directory in a .npy file. To be able to read this in C++, there is an additional .hpp file ([npy2apintstream.hpp](https://github.com/Xilinx/finn/blob/main/src/finn/qnn-data/cpp/npy2apintstream.hpp)) in FINN, which uses cnpy to read .npy files and convert them into streams, or to read a stream and write it into an .npy. [cnpy](https://github.com/rogersce/cnpy) is a helper to read and write .npy and .npz formates in C++.

The result is again compared to the "golden" output.

In [ ]:
parent_model = ModelWrapper(build_dir+"/tfc_w1_a1_dataflow_parent.onnx")
sdp_node = parent_model.graph.node[1]
child_model = build_dir + "/tfc_w1_a1_for_cppsim.onnx"
getCustomOp(sdp_node).set_nodeattr("model", child_model)
output_dict = oxe.execute_onnx(parent_model, input_dict)
output_cppsim = output_dict[list(output_dict.keys())[0]]

try:
    assert np.isclose(output_cppsim, np.where(output_golden[0]==np.amax(output_golden[0])), atol=1e-3).all()
    print("Results are the same!")
except AssertionError:
    assert False, "The results are not the same!"

## Emulation (rtlsim) using PyVerilator

The emulation using [PyVerilator](https://github.com/maltanar/pyverilator) can be done after IP blocks are generated from the corresponding HLS layers or for RTL layers directly using the generated Verilog files. Pyverilator is a tool which makes it possible to simulate verilog files using verilator via a python interface.

We have two ways to use rtlsim, one is to run the model node-by-node as with the simulation methods, but if the model is in the form of the dataflow partition, the part of the graph that consist of only HLS/RTL nodes could also be executed as whole.

Because at the point where we want to grab and verify the model, the model is already in split form (parent graph consisting of non-hls layers and child graph consisting only of hls layers) we first have to reference the child graph within the parent graph. This is done using the node attribute `model` for the `StreamingDataflowPartition` node.

First the procedure is shown, if the child graph has ip blocks corresponding to the individual layers, then the procedure is shown, if the child graph already has a stitched IP.

### Emulation of model node-by-node

The child model is loaded and the `exec_mode` for each node is set. To prepare the node-by-node emulation the transformation `PrepareRTLSim` is applied to the child model. With this transformation the emulation files are created for each node and can be used directly when calling `execute_onnx()`. Each node has a new node attribute "rtlsim_so" after transformation, which contains the path to the corresponding emulation files. Then it is saved in a new .onnx file so that the changed model can be referenced in the parent model.

In [ ]:
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP

test_fpga_part = "xc7z020clg400-1"
target_clk_ns = 10

child_model = ModelWrapper(build_dir + "/tfc_w1_a1_set_folding_factors.onnx")
child_model = child_model.transform(GiveUniqueNodeNames())
child_model = child_model.transform(PrepareIP(test_fpga_part, target_clk_ns))
child_model = child_model.transform(HLSSynthIP())
child_model = child_model.transform(SetExecMode("rtlsim"))
child_model = child_model.transform(PrepareRTLSim())
child_model.save(build_dir + "/tfc_w1_a1_dataflow_child.onnx")

The next step is to load the parent model and set the node attribute `model` in the StreamingDataflowPartition node (`sdp_node`). Afterwards the `exec_mode` is set in the parent model in each node and the model can be executed.

In [ ]:
# parent model
model_for_rtlsim = ModelWrapper(build_dir + "/tfc_w1_a1_dataflow_parent.onnx")
# reference child model
sdp_node = getCustomOp(model_for_rtlsim.graph.node[1])
sdp_node.set_nodeattr("model", build_dir + "/tfc_w1_a1_dataflow_child.onnx")

model_for_rtlsim = model_for_rtlsim.transform(SetExecMode("rtlsim"))

In [ ]:
output_dict = oxe.execute_onnx(model_for_rtlsim, input_dict)
output_rtlsim = output_dict[list(output_dict.keys())[0]]

try:
    assert np.isclose(output_rtlsim, np.where(output_golden[0]==np.amax(output_golden[0])), atol=1e-3).all()
    print("Results are the same!")
except AssertionError:
    assert False, "The results are not the same!"

### Emulation of stitched IP

Here we use the same procedure. First the child model is loaded, but in contrast to the layer-by-layer emulation, the metadata property `exec_mode` is set to "rtlsim" for the whole child model. When the model is integrated and executed in the last step, the verilog files of the stitched IP of the child model are used.

In [ ]:
from finn.transformation.fpgadataflow.insert_dwc import InsertDWC
from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP

child_model = ModelWrapper(build_dir + "/tfc_w1_a1_dataflow_child.onnx")
child_model = child_model.transform(InsertDWC())  
child_model = child_model.transform(InsertFIFO(create_shallow_fifos=True))
# DWC and FIFOs need to be specialized to either HLS or RTL variants
child_model = child_model.transform(SpecializeLayers(test_fpga_part))
child_model.save(build_dir + "/test.onnx");
child_model = child_model.transform(GiveUniqueNodeNames())
child_model = child_model.transform(PrepareIP(test_fpga_part, target_clk_ns))
child_model = child_model.transform(HLSSynthIP())
child_model = child_model.transform(CreateStitchedIP(test_fpga_part, target_clk_ns))
child_model = child_model.transform(PrepareRTLSim())
child_model.set_metadata_prop("exec_mode","rtlsim")
child_model.save(build_dir + "/tfc_w1_a1_dataflow_child.onnx");

In [ ]:
# parent model
model_for_rtlsim = ModelWrapper(build_dir + "/tfc_w1_a1_dataflow_parent.onnx")
# reference child model
sdp_node = getCustomOp(model_for_rtlsim.graph.node[1])
sdp_node.set_nodeattr("model", build_dir + "/tfc_w1_a1_dataflow_child.onnx")

In [ ]:
output_dict = oxe.execute_onnx(model_for_rtlsim, input_dict)
output_rtlsim = output_dict[list(output_dict.keys())[0]]

In [ ]:
try:
    assert np.isclose(output_rtlsim, np.where(output_golden[0]==np.amax(output_golden[0])), atol=1e-3).all()
    print("Results are the same!")
except AssertionError:
    assert False, "The results are not the same!"